In [ ]:
import pywikibot
from csv import DictWriter
import json

In [ ]:
path = "../data/categories"
fieldnames = [
    "FAMILY",
    "LOCALITY",
    "CATEGORY"
]

In [ ]:
with open(f"{path}/categories.json", "r+") as categories_raw, open(f"{path}/cat-list.csv", "a") as pages_list:
    dictwriter = DictWriter(pages_list, fieldnames, lineterminator="\r")
    categories = json.load(categories_raw)

    for idx, wiki in enumerate(categories):
        cat_pending = list(set(wiki["categories"]) - set(wiki["added"]))

        site = pywikibot.Site(wiki["locality"], wiki["family"])

        cat_obj_list: list[pywikibot.Category] = list(map(
            lambda cat:
                pywikibot.Category(site, cat)
        , cat_pending))

        for category in cat_obj_list:
            print(f"Querying {category}")

            sub_cats = category.subcategories(recurse=True)

            sub_cats = list(map(
                lambda cat:
                    {
                        "FAMILY": wiki["family"],
                        "LOCALITY": wiki["locality"],
                        "CATEGORY": cat.title()
                    }
            , [category, *sub_cats]))

            dictwriter.writerows(sub_cats)

            wiki["added"].insert(-1, category.title().replace("Category:", ""))
            categories[idx] = wiki

            categories_raw.seek(0)
            json.dump(categories, categories_raw, indent=4)